In [ ]:
%matplotlib widget
import numpy as np

theta = np.pi/6
phi = np.pi/3
euler = np.array([theta, phi, 0.0])

rotmat = np.array([[0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [-1.0, 0.0, 0.0]])

# visualize rotation matrix
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.quiver(0, 0, 0, rotmat[0, 0], rotmat[0, 1], rotmat[0, 2], color='r')
ax.quiver(0, 0, 0, rotmat[1, 0], rotmat[1, 1], rotmat[1, 2], color='g')
ax.quiver(0, 0, 0, rotmat[2, 0], rotmat[2, 1], rotmat[2, 2], color='b')
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])

In [4]:
import torch
import numpy as np
def rpy2rotmat(rpy: torch.Tensor) -> torch.Tensor:
    # convert roll-pitch-yaw to rotation matrix with torch
    # rpy: (batch_size, 3)
    # rotmat: (batch_size, 3, 3)
    roll, pitch, yaw = rpy[:, 0], rpy[:, 1], rpy[:, 2]
    rotmat = torch.zeros(rpy.shape[0], 3, 3, device=rpy.device)
    rotmat[:, 0, 0] = torch.cos(yaw) * torch.cos(pitch)
    rotmat[:, 0, 1] = torch.cos(yaw) * torch.sin(pitch) * torch.sin(roll) - torch.sin(yaw) * torch.cos(roll)
    rotmat[:, 0, 2] = torch.cos(yaw) * torch.sin(pitch) * torch.cos(roll) + torch.sin(yaw) * torch.sin(roll)
    rotmat[:, 1, 0] = torch.sin(yaw) * torch.cos(pitch)
    rotmat[:, 1, 1] = torch.sin(yaw) * torch.sin(pitch) * torch.sin(roll) + torch.cos(yaw) * torch.cos(roll)
    rotmat[:, 1, 2] = torch.sin(yaw) * torch.sin(pitch) * torch.cos(roll) - torch.cos(yaw) * torch.sin(roll)
    rotmat[:, 2, 0] = -torch.sin(pitch)
    rotmat[:, 2, 1] = torch.cos(pitch) * torch.sin(roll)
    rotmat[:, 2, 2] = torch.cos(pitch) * torch.cos(roll)
    return rotmat

In [16]:
angle = 0.0
euler = np.array([angle, 0.0, 0.0])
rpy = torch.tensor(euler).unsqueeze(0)
rotmat = rpy2rotmat(rpy)

In [17]:
force = torch.tensor([0.0, 0.0, 1.0])
(rotmat[0]@force)[2] - np.cos(angle)

tensor(0.)